In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 70)
pd.set_option('display.max_rows',150)

In [3]:
df = pd.read_csv('input/cierres_nc/210813-1716-cierres_nc.csv', sep=';', dtype='object')

In [4]:
f4 = pd.read_csv('input/cierres_nc/210813-1716-f4.csv', sep=';', dtype='object')

In [5]:
df.shape

(68372, 15)

In [9]:
cerrado = df[df['estado_final']=='cerrado']

In [10]:
cerrado.shape

(62703, 15)

In [11]:
eac = cerrado[cerrado['tipificacion_final']=='se asocia f4 dado de baja por producto entregado a cliente'].reset_index(drop=True)

In [12]:
eac.shape

(446, 15)

Unir f4 y cnc

In [13]:
ef = eac.merge(f4, how='left', left_on=['f4', 'upc'], right_on=['nro_red_inventario', 'upc'])

In [14]:
ef.loc[ef.nro_red_inventario.isna()].shape

(0, 22)

Tiene # de F4?

In [15]:
na = ef.loc[ef.f4.isna()]
ef.loc[na.index,'D2'] = 'No tiene número de F4'
notna = ef.loc[ef.f4.notna()]

F4 único 

In [16]:
ef.loc[(ef.D2.isna())& (ef.duplicated(['f4'], keep=False))].shape

(168, 23)

In [17]:
ef.loc[(ef.D2.isna())& (ef.duplicated(['f4'], keep=False)), 'D2'] = 'F4 tiene varias NC'

Fecha de creación diferente a julio

In [18]:
ef.fecha_creacion.unique()

array(['12-jul-2021'], dtype=object)

In [19]:
ef.loc[(ef.D2.isna())& (ef.fecha_creacion=='05-may-2021')].shape 

(0, 23)

In [17]:
ef.loc[(ef.D2.isna())& (ef.fecha_creacion=='05-may-2021') ,'D2'] = 'F4 con fecha de creación diferente a 07/21'

In [20]:
ef.loc[(ef.D2.isna())& (ef.fecha_creacion=='11-jun-2021')].shape 

(0, 23)

In [19]:
ef.loc[(ef.D2.isna())&(ef.fecha_creacion=='11-jun-2021'), 'D2'] = 'F4 con fecha de creación diferente a 07/21'

F4 anulados

In [21]:
ef.loc[ef.D2.isna()]['estado'].unique()

array(['anulado', 'registrado'], dtype=object)

In [22]:
ef.loc[(ef.D2.isna())&(ef.estado=='anulado')].shape

(266, 23)

In [23]:
ef.loc[(ef.D2.isna())&(ef.estado=='anulado'), 'D2'] = 'F4 con estado anulado'

Existen F4 en estado registrado? 

In [24]:
ef.loc[(ef.D2.isna())&(ef.estado=='registrado')].shape

(12, 23)

In [25]:
ncc = ef.loc[(ef.D2.isna())&(ef.cantidad_trx_actual!=ef.cantidad)]

In [26]:
ncc.to_excel(f'output/210818-d2-f4-ncc.xlsx', sheet_name=f'nc-f4-base', index=False) 

In [27]:
ef.head()

,cod_aut_nc,local_trx,terminal,local_ant,upc,sku,cantidad_trx_actual,ct,tipo_nc,f5,f11_x,f3,f4,estado_final,tipificacion_final,nro_red_inventario,estado,fecha_creacion,destino,linea,cantidad,f11_y,D2
0,1060774658,3000,81,2000,2003681706009,3681706,1,49748,PARCIAL,NaN,NaN,NaN,46324658,cerrado,se asocia f4 dado de baja por producto entrega...,46324658,anulado,12-jul-2021,12603349417 cod aut 1060774658,j07,1,12603349417,F4 con estado anulado
1,1060740363,3000,17,2000,2003431020003,3431020,1,94118,PARCIAL,NaN,NaN,NaN,46324672,cerrado,se asocia f4 dado de baja por producto entrega...,46324672,anulado,12-jul-2021,12624214794 cod aut 1060740363,j08,1,12624214794,F4 con estado anulado
2,1060739329,3000,17,2000,2003558344006,3558344,1,227899,PARCIAL,NaN,NaN,NaN,46331045,cerrado,se asocia f4 dado de baja por producto entrega...,46331045,anulado,12-jul-2021,12623786148 cod aut 1060739329,j08,1,12623786148,F4 con estado anulado
3,1060388084,3000,81,2000,7701144157775,3324151,1,605035,PARCIAL,NaN,NaN,NaN,46331076,cerrado,se asocia f4 dado de baja por producto entrega...,46331076,anulado,12-jul-2021,12624127667 cod aut 1060388084,j14,1,12624127667,F4 con estado anulado
4,1060982343,3000,3001,2000,2005243666004,5243666,1,53748,PARCIAL,NaN,NaN,NaN,46324702,cerrado,se asocia f4 dado de baja por producto entrega...,46324702,anulado,12-jul-2021,12597813499 cod aut 1060982343,j07,1,12597813499,F4 con estado anulado


In [28]:
ef.loc[ef.D2.isna(), 'D2'] = 'OK'

In [29]:
list(ef.loc[(ef.D2.isna()), 'destino'])

[]

In [30]:
ef['ct'] = pd.to_numeric(ef['ct'])

In [32]:
ef.to_excel(f'output/210818-d2-f4.xlsx', sheet_name=f'nc-f4-base', index=False) 